In [2]:
import pandas as pd
import numpy as np

### Obtaining and cleaning data

In [3]:
df = pd.read_csv("entries.csv")

In [6]:
print(len(df))
print(df.head())

17067
   id                            andrew_id_hash semester           entry_time  \
0   1  5660915b416b110d3125eddaf15d004c09f2dcf9      S17  2017-01-18 16:30:42   
1   2  7a3ff1726a9e59878e74b78b612e705ba388f35c      S17  2017-01-18 16:30:47   
2   3  33573dcedddbc5924f4fc0c16db8b901fd1c8286      S17  2017-01-18 16:31:00   
3   4  2291bad572687d3d7e8fb1c4c4ad21c0b83c1e67      S17  2017-01-18 16:31:09   
4   5  9a08eb5de601714dce83c7c7e5040509349f512c      S17  2017-01-18 16:32:51   

             help_time            exit_time  wait_estimate  topic_id  ta_id  
0  2017-01-18 16:31:36  2017-01-18 16:31:39            NaN       5.0      8  
1  2017-01-18 16:31:44  2017-01-18 16:34:53            NaN       4.0     22  
2  2017-01-18 16:32:12  2017-01-18 16:33:05            NaN       4.0     17  
3  2017-01-18 16:32:28  2017-01-18 16:38:52            NaN       4.0     16  
4  2017-01-18 16:33:15  2017-01-18 16:39:42            0.0       1.0     17  


In [17]:
df2 = df.drop(['wait_estimate', 'ta_id', 'id', 'semester'], axis=1)
df2 = df3.dropna(axis=0, how='any')
print(len(df2))

16062


In [20]:
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16062 entries, 0 to 17066
Data columns (total 5 columns):
andrew_id_hash    16062 non-null object
entry_time        16062 non-null object
help_time         16062 non-null object
exit_time         16062 non-null object
topic_id          16062 non-null float64
dtypes: float64(1), object(4)
memory usage: 752.9+ KB
None


In [23]:
# Do not execute again
df2['entry_time'] = pd.to_datetime(df2['entry_time'])
df2['help_time'] = pd.to_datetime(df2['help_time'])
df2['exit_time'] = pd.to_datetime(df2['exit_time'])

In [24]:
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16062 entries, 0 to 17066
Data columns (total 5 columns):
andrew_id_hash    16062 non-null object
entry_time        16062 non-null datetime64[ns]
help_time         16062 non-null datetime64[ns]
exit_time         16062 non-null datetime64[ns]
topic_id          16062 non-null float64
dtypes: datetime64[ns](3), float64(1), object(1)
memory usage: 752.9+ KB
None


In [25]:
df2['WT'] = (df2['help_time'] - df2['entry_time']).astype('timedelta64[s]')
df2['HT'] = (df2['exit_time'] - df2['help_time']).astype('timedelta64[s]')

In [28]:
df3 = df2[['andrew_id_hash', 'WT', 'HT', 'topic_id']]
print(df3.info())
print(df3.head())
print(len(df3))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16062 entries, 0 to 17066
Data columns (total 4 columns):
andrew_id_hash    16062 non-null object
WT                16062 non-null float64
HT                16062 non-null float64
topic_id          16062 non-null float64
dtypes: float64(3), object(1)
memory usage: 1.2+ MB
None
                             andrew_id_hash    WT     HT  topic_id
0  5660915b416b110d3125eddaf15d004c09f2dcf9  54.0    3.0       5.0
1  7a3ff1726a9e59878e74b78b612e705ba388f35c  57.0  189.0       4.0
2  33573dcedddbc5924f4fc0c16db8b901fd1c8286  72.0   53.0       4.0
3  2291bad572687d3d7e8fb1c4c4ad21c0b83c1e67  79.0  384.0       4.0
4  9a08eb5de601714dce83c7c7e5040509349f512c  24.0  387.0       1.0
16062
